In [ ]:
from matplotlib import pyplot
import json
import numpy as np

%matplotlib inline

In [ ]:
def load_route(route, reverse=False):
    if reverse:
        data_file = 'route_data/{}_data.json'.format(route)
    else:
        data_file = 'route_data/{}_reverse_data.json'.format(route)
    return json.load(open(data_file, 'r'))

In [ ]:
# define simulation constants

ROUTES = [10, 11, 15, 17, 81, 82]

# physical constants
GRAVITY              = 9.81  # [m/s^2]
FUEL_DENSITY         = 35800 # [kJ/L]
AIR_DENSITY          = 1.225 # [kg/m^3]

# bus specifications
MASS                 = 9500  # [kg]
CROSS_AREA           = 7.63  # [m^2]

# model parameters
parameters = {
    'DRAG_COEFFICIENT'     : 0.70,  # []
    'ROLLING_COEFFICIENT'  : 0.012, # []
    'ENGINE_EFFICIENCY'    : 0.35,  # []
    'BRAKING_EFFICIENCY'   : 0.50,  # []
    'CHARGING_EFFICIENCY'  : 0.65,  # []
    'CAPACITY'             : 44600, # [kj]
    'MAX_CHARGE_RATE'      : 37,    # [kW]
    'MAX_DISCHARGE_RATE'   : 99,    # [kW]
    'ENGINE_CHARGE_RATE'   : 1,     # [kW]
    'BRAKING_ACCEL'        : 1.5,   # [m/s^2]
    'FORWARD_ACCEL'        : 1.1,   # [m/s^2]
    'IDLE_FUEL_USE'        : 3.93,  # [L/h]
    'STOP_TIME'            : 12,    # [s]
}

In [ ]:
# run simulation

def run_simulation(route, params, hybrid, initial_battery, time_limit, dt, reverse, winter):
    path = load_route(route, reverse)
    
    position = 0
    velocity = 0
    time_stopped = 0
    fuel_consumed = 0
    battery = initial_battery
    total_regen = 0

    output = []
    stop_times = []
    last_stop = None

    for t in np.arange(0, time_limit * 60, dt):
        if position > path[-1]['dist']:
            break
            
        index = len([p for p in path if p['dist'] < position])
        next_point = path[index]
        last_point = path[index - 1]
        
        stop_offset = 3
        if next_point['stop'] and next_point['dist'] - position < stop_offset*1.1 and last_stop != index - 1:
            stop_times.append(t / 60)
            last_stop = index - 1

        speed_limit = 15.6
        gradient = last_point['gradient']
        elevation = (last_point['elevation'] if index == 0
                else last_point['elevation'] + (position - last_point['dist']) * gradient)

        future_stops = [p['dist'] for p in path[index:] if p['stop']]
        dist_to_stop = 1e9 if len(future_stops) == 0 else future_stops[0] - position
        stopping_time = velocity / params['BRAKING_ACCEL']
        stopping_dist = 0.5 * velocity * stopping_time
        
        # stop and wait if at a stop
        if dist_to_stop < stop_offset and time_stopped < params['STOP_TIME']:
            if velocity > params['BRAKING_ACCEL'] * dt:
                acceleration = -params['BRAKING_ACCEL']
            else:
                acceleration = -velocity * dt
            time_stopped += dt

        # slow down if approaching a stop
        elif stopping_dist + stop_offset >= dist_to_stop and time_stopped < params['STOP_TIME']:
            acceleration = -params['BRAKING_ACCEL']

        # speed up if not approaching a stop
        elif velocity < speed_limit:
            acceleration = params['FORWARD_ACCEL']

        # maintain speed
        else:
            acceleration = 0
            
        if dist_to_stop > stop_offset:
            time_stopped = 0

        if winter and gradient > 0.05:
            slippage_factor = 1 / 0.8
        else:
            slippage_factor = 1
            
        power = 0.001 * (
                MASS * GRAVITY * gradient * velocity + # gravitational
                MASS * velocity * acceleration * slippage_factor + # accelerational
                0.5 * params['DRAG_COEFFICIENT'] * AIR_DENSITY * CROSS_AREA * velocity ** 3 + # air drag
                MASS * GRAVITY * params['ROLLING_COEFFICIENT'] * velocity # rolling drag
        )

        if hybrid:
            real_drate = min(battery / dt, params['MAX_DISCHARGE_RATE'])
            real_crate = min((params['CAPACITY'] - battery) / dt, params['MAX_CHARGE_RATE'])
            battery_power = min(real_drate, max(-real_crate, power))

            if battery >= params['CAPACITY']:
                battery_power = max(battery_power, 0)

            if battery <= 0:
                battery_power = min(battery_power, 0)

            if battery_power > 0: # discharging
                battery -= battery_power * dt
            else: # charging
                battery -= battery_power * params['BRAKING_EFFICIENCY'] * dt
                total_regen -= battery_power * params['BRAKING_EFFICIENCY'] * dt

            if power > battery_power:
                engine_power = power - battery_power
            else:
                engine_power = 0

            if engine_power > 0 and battery < params['CAPACITY']:
                engine_power += params['ENGINE_CHARGE_RATE']
                battery += params['ENGINE_CHARGE_RATE'] * params['CHARGING_EFFICIENCY'] * dt

        else:
            battery_power = 0
            engine_power = max(power, 0)
            
        idle_drain = params['IDLE_FUEL_USE'] / 3600 * FUEL_DENSITY * params['ENGINE_EFFICIENCY']
        engine_power = max(idle_drain, engine_power)
        
        fuel_consumed += engine_power / params['ENGINE_EFFICIENCY'] / FUEL_DENSITY * dt

        velocity += acceleration * dt
        position += velocity * dt

        output.append([
                ['Time [min]', t / 60], 
                ['Velocity [m/s]', velocity], 
                ['Elevation [m]', elevation],
                ['Gradient', gradient], 
                ['Power [kW]', power], 
                ['Battery Power [kW]', battery_power], 
                ['Energy Stored [MJ]', battery / 1000],
                ['Total Regen [MJ]', total_regen / 1000],
                ['Engine Power [kW]', engine_power], 
                ['Fuel Consumed [L]', fuel_consumed]])
        
    return {
        'fuel_consumed': fuel_consumed, 
        'final_battery': battery, 
        'total_regen': total_regen / 1000,
        'stop_times': stop_times,
        'total_time': output[-1][0][1],
        'total_dist': position,
        'data': np.array(output)
    }

In [ ]:
def find_steady_state(data, margin=100):
    initial_battery = 0
    while True:
        sim = run_simulation(data, hybrid=True, initial_battery=initial_battery)
        battery = sim['final_battery']
        if battery - margin <= initial_battery <= battery + margin:
            return battery
        initial_battery = battery

In [ ]:
def render_simulation(route, parameters, hybrid, initial_battery=0, time_limit=60, dt=0.05, reverse=False, winter=False):
    sim = run_simulation(route, parameters, hybrid=hybrid, initial_battery=0, time_limit=60, dt=dt, reverse=reverse, winter=winter)
    print('fuel rate: {:.2f} l/h'.format(sim['fuel_consumed'] / sim['total_time'] * 60))
    kpl = sim['total_dist'] / 1000 / sim['fuel_consumed']
    print('efficiency: {:.2f} km/l ({:.2f} mpg)'.format(kpl, kpl * 2.35))
    print('distance: {:.2f} km'.format(sim['total_dist'] / 1000))
    print('time: {:.2f}'.format(sim['total_time']))
    print('ending battery: {:.0f} kJ'.format(sim['final_battery']))
    print('total regen: {:.2f} MJ'.format(sim['total_regen']))


    output = sim['data']
    channels = output.shape[1] - 1
    show_channels = [0, 1, 2, 3, 4, 5, 6, 7, 8]

    tlim = (4.61, 6)

    f, axes = pyplot.subplots(len(show_channels), figsize=(15, 3 * len(show_channels)))
    for i, channel in enumerate(show_channels):
        if len(show_channels) == 1:
            axis = axes
        else:
            axis = axes[i]
        axis.plot(output[int(tlim[0]*60/dt):int(tlim[1]*60/dt), 0, 1], 
                           output[int(tlim[0]*60/dt):int(tlim[1]*60/dt), channel + 1, 1])
        axis.set(ylabel=output[0, channel + 1, 0])
        for t in [t for t in sim['stop_times'] if tlim[0] < t < tlim[1]]:
            axis.axvline(t, color='r')

In [ ]:
render_simulation(10, parameters, True)

In [ ]:
def parameter_partial(parameter_name, relative_epsilon=-.01):
    alt_params = {name: parameter for name, parameter in parameters.items()}
    alt_params[parameter_name] = parameters[parameter_name] * (1 + relative_epsilon)
        
    base_improvements = []
    alt_improvements = []
    
    for route in ROUTES:
        base_diesel = run_simulation(route, parameters, False, initial_battery=0, time_limit=60, dt=0.05, reverse=False, winter=False)
        base_hybrid = run_simulation(route, parameters, True, initial_battery=0, time_limit=60, dt=0.05, reverse=False, winter=False)
        alt_diesel = run_simulation(route, alt_params, False, initial_battery=0, time_limit=60, dt=0.05, reverse=False, winter=False)
        alt_hybrid = run_simulation(route, alt_params, True, initial_battery=0, time_limit=60, dt=0.05, reverse=False, winter=False)
        
        base_diesel_rate = base_diesel['fuel_consumed'] / base_diesel['total_time']
        base_hybrid_rate = base_hybrid['fuel_consumed'] / base_hybrid['total_time']
        alt_diesel_rate = alt_diesel['fuel_consumed'] / alt_diesel['total_time']
        alt_hybrid_rate = alt_hybrid['fuel_consumed'] / alt_hybrid['total_time']
        
        base_improvements.append(base_diesel_rate - base_hybrid_rate)
        alt_improvements.append(alt_diesel_rate - alt_hybrid_rate)
    
#     print(base_improvements, alt_improvements)
        
    absolute_delta = 0
    for bi, ai in zip(base_improvements, alt_improvements):
        absolute_delta += abs((bi - ai) / bi)
        
#     print(absolute_delta)
    
    return absolute_delta / relative_epsilon / len(ROUTES)

In [ ]:
for p in sorted(parameters.keys()):
    print('{:<25} {:.3f}'.format(p, parameter_partial(p)))

In [ ]:
winter = True

for route in ROUTES:
    diesel = run_simulation(route, parameters, False, 0, 60, 0.05, False, winter=winter)
    h1 = run_simulation(route, parameters, True, 0, 60, 0.05, False, winter=winter)
    hybrid = run_simulation(route, parameters, True, h1['final_battery'], 60, 0.05, False, winter=winter)
    print('Route {}: Diesel: {:.2f} L/h, Hybrid: {:.2f} L/h'.format(
            route, diesel['fuel_consumed'] / diesel['total_time'] * 60, hybrid['fuel_consumed'] / hybrid['total_time'] * 60))

In [ ]:
efficiencies = []
stop_rates = []
regen_rates = []

for route in routes:
    data_file = 'route_data/{}_data.json'.format(route)
    data = json.load(open(data_file, 'r'))
    diesel = run_simulation(data, hybrid=False, time_limit=120)
    steady_battery = find_steady_state(data)
    print(steady_battery)
    hybrid = run_simulation(data, hybrid=True, initial_battery=steady_battery, time_limit=120)
    stop_rate = len([p for p in data if p['stop']]) / hybrid['total_time'] * 60
    diesel_rate = diesel['fuel_consumed'] / diesel['total_time'] * 60
    hybrid_rate = hybrid['fuel_consumed'] / hybrid['total_time'] * 60
    improvement = (diesel_rate - hybrid_rate) / diesel_rate * 100
    regen_rate = hybrid['total_regen'] / hybrid['total_time'] / 60 * 1000
    print('Route {}'.format(route))
    print('diesel: {:.2f}, hybrid: {:.2f}, improvement: {:.2f}%, regen rate: {:.2f} kW, stops: {:.2f}/h'.format(
            diesel_rate, hybrid_rate, improvement, regen_rate, stop_rate))

In [ ]:
y = [6.38, 6.17, 6.12, 6.02, 5.67, 5.98]
x = [5.14, 4.52, 4.14, 4.13, 3.98, 4.38]
s = [89.0, 92.4, 100.0, 105.6, 89.4, 87.5]
pyplot.bar(list(range(len(routes))), y, align='center')
pyplot.xticks(list(range(len(routes))), [str(r) for r in routes])

In [ ]:
# improvements = []
# stop_rates = []
avg_gradient = []

for route in ROUTES:
    print(route)
    
    data = load_route(route)
    gradient = 0
    for i in range(len(data) - 1):
        gradient += abs(data[i]['gradient']) * (data[i + 1]['dist'] - data[i]['dist'])
    avg_gradient.append(gradient / data[-1]['dist'])
    
#     stops = len([p for p in data if p['stop']])
#     diesel = run_simulation(route, parameters, False, 0, 60, 0.05, False, False)
#     h1 = run_simulation(route, parameters, True, 0, 60, 0.05, False, False)
#     hybrid = run_simulation(route, parameters, True, h1['final_battery'], 60, 0.05, False, False)
#     diesel_rate = diesel['fuel_consumed'] / diesel['total_time'] * 60
#     hybrid_rate = hybrid['fuel_consumed'] / hybrid['total_time'] * 60
#     improvements.append(diesel_rate - hybrid_rate)
#     stop_rates.append(hybrid['total_time'] / stops)

In [ ]:
pyplot.scatter(stop_rates, improvements)

In [ ]:
np.savetxt('out.csv', np.array(list(zip(improvements, stop_rates, avg_gradient))), delimiter=',')

In [ ]:
','.join(improvements)

In [ ]:
improvements